In [8]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import koreanize_matplotlib
import seaborn as sns 
from tensorflow.keras.layers import Dense,Flatten,Embedding
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.text import Tokenizer,text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from numpy import array

In [11]:
text = '해보지 않으면 해낼 수 없다'



In [14]:
result = text_to_word_sequence(text)

In [15]:
print(f"원본 : {text}")
print(f"토큰화 : {result}") # 토큰, 코퍼스(corpus) : 자연 언어 연구를 위해 

원본 : 해보지 않으면 해낼 수 없다
토큰화 : ['해보지', '않으면', '해낼', '수', '없다']


In [16]:
# 단어 빈도수 세기 
docs = ["먼저 텍스트의 각 단어를 나누어 토큰화 합니다.",
       "텍스트의 단어로 토큰화 해야 딥러닝에서 인식",
       "토큰화한 결과는 딥러닝에서 사용할 수 있습니다"]

In [17]:
token = Tokenizer(lower=False)
token.fit_on_texts(docs)

In [18]:
print(f"단어 갯수 : {token.word_counts}")

단어 갯수 : OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 2), ('합니다', 1), ('단어로', 1), ('해야', 1), ('딥러닝에서', 2), ('인식', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])


In [19]:
print(f"문장 카운트 {token.document_count}")

문장 카운트 3


In [20]:
print(f"각 단어가 몇 개의 문장에 포함 되어 있는가?{token.word_docs}")

각 단어가 몇 개의 문장에 포함 되어 있는가?defaultdict(<class 'int'>, {'각': 1, '토큰화': 2, '텍스트의': 2, '나누어': 1, '합니다': 1, '단어를': 1, '먼저': 1, '딥러닝에서': 2, '해야': 1, '단어로': 1, '인식': 1, '토큰화한': 1, '있습니다': 1, '수': 1, '결과는': 1, '사용할': 1})


In [21]:
print(f"각 단어에 매겨진 인덱스 :{token.word_index}")

각 단어에 매겨진 인덱스 :{'텍스트의': 1, '토큰화': 2, '딥러닝에서': 3, '먼저': 4, '각': 5, '단어를': 6, '나누어': 7, '합니다': 8, '단어로': 9, '해야': 10, '인식': 11, '토큰화한': 12, '결과는': 13, '사용할': 14, '수': 15, '있습니다': 16}


## 단어의 원 핫 인코딩

In [22]:
text = "오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"
token = Tokenizer(lower=False)
token.fit_on_texts([text])

print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [25]:
# 단어를 벡터로 변환 

x = token.texts_to_sequences([text])

In [26]:
# 인덱스에 1을 추가해서 원 - 핫 인코딩 배열을 만들기

word_size = len(token.word_index) +1 
# 

In [27]:
x = to_categorical(x,num_classes=word_size)
print(x)

# 원 핫 인코딩 행렬

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


## 텍스트를 읽고 긍정, 부정 예측하기 

In [62]:
docs = ["너무 재미있네요","최고에요","참 잘 만든 영화예요","추천하고 싶은 영화 입니다.",
       "한번 더 보고싶네요","글쎄요","별로에요","생각보다 지루하네요","연기가 어색해요","재미없어요",
       ]

# 긍정 리뷰 1 , 부정 리뷰 0으로 클래스 지정 
classes = np.array([1,1,1,1,1,0,0,0,0,0])

In [34]:
# 토큰화 

token = Tokenizer(lower=False)
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재미있네요': 2, '최고에요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화': 10, '입니다': 11, '한번': 12, '더': 13, '보고싶네요': 14, '글쎄요': 15, '별로에요': 16, '생각보다': 17, '지루하네요': 18, '연기가': 19, '어색해요': 20, '재미없어요': 21}


In [36]:
# 원 핫 인코딩 

y = token.texts_to_sequences(docs)

In [46]:
# 패딩 추가 
max_length = max(len(item) for item in y)

In [47]:
padded_x = pad_sequences(y,max_length)

In [55]:
padded_x[1]

array([0, 0, 0, 3])

In [59]:
classes[1]

1

In [49]:
# 임베딩에 입력된 단어의 수를 지정 
word_size = len(token.word_index)+ 1 

In [50]:
# 단어 임베딩을 포함해서 딥러닝 모델 생성 
model = Sequential()
model.add(Embedding(word_size,8,input_length=max_length))
model.add(Flatten())
model.add(Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              176       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________


In [60]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
history = model.fit(padded_x,classes,epochs=20)
print(f"Accuracy: {model.evaluate(padded_x,classes)[1]}")

Epoch 1/20
1/1 [==============================] - 0s 411ms/step - loss: 0.5956 - accuracy: 0.9000
Epoch 2/20
1/1 [==============================] - 0s 20ms/step - loss: 0.5931 - accuracy: 0.9000
Epoch 3/20
1/1 [==============================] - 0s 17ms/step - loss: 0.5906 - accuracy: 0.9000
Epoch 4/20
1/1 [==============================] - 0s 1ms/step - loss: 0.5881 - accuracy: 0.9000
Epoch 5/20
1/1 [==============================] - 0s 0s/step - loss: 0.5856 - accuracy: 0.9000
Epoch 6/20
1/1 [==============================] - 0s 0s/step - loss: 0.5831 - accuracy: 0.9000
Epoch 7/20
1/1 [==============================] - 0s 0s/step - loss: 0.5805 - accuracy: 0.9000
Epoch 8/20
1/1 [==============================] - 0s 13ms/step - loss: 0.5780 - accuracy: 0.9000
Epoch 9/20
1/1 [==============================] - 0s 0s/step - loss: 0.5755 - accuracy: 0.9000
Epoch 10/20
1/1 [==============================] - 0s 0s/step - loss: 0.5729 - accuracy: 0.9000
Epoch 11/20
1/1 [======================

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, <class 'numpy.int32'>